# MS1: Make a Data Extraction notebook.
- Debugging dataset: small enough to test code with; reasonable code should run in 2 minutes
- Working dataset:  large enough to do the problem on (training should run no more than 40 minutes)
- Convert these datasets to pandas
  - I suggest that you convert datetime to pandas timestamps (allows for time deltas and time manipulation)
  - Pickle (https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_pickle.html)  the data.  
  - That converts it to a binary file which can be loaded directly (must faster) into the correct datatypes


In [ ]:
#imports
import pandas as pd
from datetime import datetime
import os
import gdown

**Downloading data from our drive**

In [ ]:
file_id = "1QFHwIlyP6CsOvtvK7KDhaA_k-7DjIvsg"
download_url = f"https://drive.google.com/uc?export=download&id={file_id}"
file = 'PERM_Disclosure_Data_FY2022_Q4.xlsx'
gdown.download(download_url, file, quiet=False)

Downloading...
From (original): https://drive.google.com/uc?export=download&id=1QFHwIlyP6CsOvtvK7KDhaA_k-7DjIvsg
From (redirected): https://drive.google.com/uc?export=download&id=1QFHwIlyP6CsOvtvK7KDhaA_k-7DjIvsg&confirm=t&uuid=8e689491-ba62-4b17-9759-bb5a2d8edc64
To: /content/PERM_Disclosure_Data_FY2022_Q4.xlsx
100%|██████████| 88.6M/88.6M [00:00<00:00, 97.4MB/s]


'PERM_Disclosure_Data_FY2022_Q4.xlsx'

In [ ]:
#read file
df = pd.read_excel(file)

**Mounting drive to save pickled data**

In [ ]:
#mount drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Creation of Debugging and Working Dataset**
- Cleaning the data by removing features with over 50% missing values.
- Removing features with excess missing values.
- Pickled at the end.


In [ ]:
#cleaning and pickeling
cleaned_full_data = '/content/drive/MyDrive/Deep Dive Group 27: Visas/cleaned_full_data.pkl'
cleaned_debugging_data = '/content/drive/MyDrive/Deep Dive Group 27: Visas/cleaned_debugging_data.pkl'
threshold = 0.5 * len(df)
df_cleaned = df.dropna(thresh=threshold, axis=1)

columns_to_remove = ['LOCAL_ETHNIC_PAPER_FROM_DATE', 'LOCAL_ETHNIC_PAPER_TO_DATE']
df_cleaned = df_cleaned.drop(columns=columns_to_remove)

def get_datetime_columns(df):
    return [col for col in df.columns if 'date' in col.lower()]
datetime_columns = get_datetime_columns(df_cleaned)
for col in datetime_columns:
    if col in df_cleaned.columns:
        df_cleaned[col] = pd.to_datetime(df_cleaned[col], errors='coerce')

df_cleaned.to_pickle(cleaned_full_data)

debugging_dataset = df_cleaned.dropna().sample(frac=0.20, random_state=42)
debugging_dataset.to_pickle(cleaned_debugging_data)

In [ ]:
print(df_cleaned.shape)
print(df_cleaned.tail())

(104600, 107)
          CASE_NUMBER CASE_STATUS RECEIVED_DATE DECISION_DATE REFILE  \
104595  A-22004-93324      Denied    2022-01-29    2022-09-30      N   
104596  A-22004-93336      Denied    2022-01-29    2022-09-30      N   
104597  A-22004-93434      Denied    2022-01-29    2022-09-30      N   
104598  A-22026-02796      Denied    2022-01-26    2022-09-30      N   
104599  A-22028-04339      Denied    2022-01-26    2022-09-30      N   

       SCHD_A_SHEEPHERDER              EMPLOYER_NAME      EMPLOYER_ADDRESS_1  \
104595                  N  Groendyke Transport, Inc.  2510 Rock Island Blvd.   
104596                  N  Groendyke Transport, Inc.  2510 Rock Island Blvd.   
104597                  N  Groendyke Transport, Inc.  2510 Rock Island Blvd.   
104598                  N       Veriha Trucking Inc.      2830 Cleveland Ave   
104599                  N        SENSOR PRODUCTS INC         300 MADISON AVE   

       EMPLOYER_CITY EMPLOYER_STATE_PROVINCE  ... FOREIGN_WORKER_ALT_OCC

In [ ]:
print(debugging_dataset.shape)
print(debugging_dataset.tail())

(5329, 107)
         CASE_NUMBER        CASE_STATUS RECEIVED_DATE DECISION_DATE REFILE  \
16448  A-21160-98688  Certified-Expired    2021-06-28    2021-11-24      N   
86565  A-21320-73535          Certified    2021-12-10    2022-07-19      N   
73043  A-21286-57735          Certified    2021-11-04    2022-05-19      N   
73881  A-21287-58619          Certified    2021-11-03    2022-05-23      N   
87155  A-21201-19569          Certified    2021-09-02    2022-07-21      N   

      SCHD_A_SHEEPHERDER                       EMPLOYER_NAME  \
16448                  N  MACYS SYSTEMS AND TECHNOLOGY, INC.   
86565                  N                    HCL AMERICA INC.   
73043                  N   TATA CONSULTANCY SERVICES LIMITED   
73881                  N                  SRS CONSULTING INC   
87155                  N               MICROSOFT CORPORATION   

                        EMPLOYER_ADDRESS_1 EMPLOYER_CITY  \
16448               5985 STATE BRIDGE ROAD   JOHNS CREEK   
86565         